# Prospecting LA's Backyard Houses with Machine Learning
## Anupama Garla  
  
## 0. Data Upload

In [7]:
#for dataframe manipulation
import pandas as pd

#get pandas and sql to work together
#import psycopg2 as pg
import pandas.io.sql as pd_sql
from psycopg2 import connect
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from sqlalchemy import create_engine

import custom helper functions
from backyard_house_helper_functions import create_psql_schema 

#set max DF column and row diplay to no limit
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
# Create dictionary to use to connect to pSQL databases
params = {
    'host': 'localhost',  # We are connecting to our _local_ version of psql
    'user': 'agar',
    #'dbname': 'properties',    # DB that we are connecting to
    'port': 5432          # port 
}

connection = pg.connect(**params)

# Connect and create properties database, disconnect, and reconnect to the right database
connection = connect(**params, 'dbname': 'votes')
connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
connection.cursor().execute('CREATE DATABASE properties;')
connection.close()

## 0.1 Read in Core Datasets

### AirBNB Datasets

In [9]:
# Read in AirBnB's properties dataset and take a look at the head
airBNB_df = pd.read_csv('listings.csv.gz', compression='gzip', 
                                 header=0, low_memory=False)
airBNB_df.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,109,https://www.airbnb.com/rooms/109,20200213032707,2020-02-13,Amazing bright elegant condo park front *UPGRA...,"*** Unit upgraded with new bamboo flooring, br...","*** Unit upgraded with new bamboo flooring, br...","*** Unit upgraded with new bamboo flooring, br...",none,NaN,NaN,NaN,NaN,NaN,Camelot NEW RESIDENTS’ GENERAL INFORMATION F...,NaN,NaN,https://a0.muscache.com/im/pictures/4321499/1d...,NaN,521,https://www.airbnb.com/users/show/521,Paolo,2008-06-27,"San Francisco, California, United States",Search for me on the Internet with the keyword...,NaN,NaN,NaN,f,https://a0.muscache.com/im/users/521/profile_p...,https://a0.muscache.com/im/users/521/profile_p...,Culver City,1.0,1.0,"['email', 'phone', 'facebook', 'reviews', 'kba']",t,t,"Culver City, CA, United States",Culver City,Culver City,Other Cities,Culver City,CA,90230,Los Angeles,"Culver City, CA",US,United States,33.98209,-118.38494,t,Condominium,Entire home/apt,6,2.0,2.0,3.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",NaN,$122.00,$904.00,"$2,851.00",$500.00,$240.00,3,$25.00,30,730,30,30,730,730,30.0,730.0,3 days ago,t,0,0,0,0,2020-02-13,2,0,2011-08-15,2016-05-15,80.0,10.0,10.0,6.0,8.0,10.0,8.0,f,NaN,"{""Culver City"","" CA""}",f,f,strict_14_with_grace_period,t,f,1,1,0,0,0.02
1,344,https://www.airbnb.com/rooms/344,20200213032707,2020-02-13,Family perfect;Pool;Near Studios!,This home is perfect for families; aspiring ch...,"Cheerful & comfortable; near studios, amusemen...",This home is perfect for families; aspiring ch...,none,Quiet-yet-close to all the fun in LA! Hollywoo...,"One dog may be on premises, friendly and cared...",Short drive to subway and elevated trains runn...,"Pool, patio and self-contained main house all ...",Host and caretaker may be available throughout...,Host asks that guests refrain from partying lo...,NaN,NaN,https://a0.muscache.com/im/pictures/cc4b724d-d...,NaN,767,https://www.airbnb.com/users/show/767,Melissa,2008-07-11,"Burbank, California, United States","Single mother, CEO and Owner of an internation...",within a day,75%,29%,f,https://a0.muscache.com/im/users/767/profile_p...,https://a0.muscache.com/im/users/767/profile_p...,Burbank,1.0,1.0,"['email', 'phone', 'reviews', 'jumio', 'kba', ...",t,t,"Burbank, CA, United 

### LA Lot Datasets

In [10]:
# Read in assessor's lot dataset and take a look at the head
properties_df = pd.read_csv("Assessor_Parcels_Data_-_2019.csv", low_memory=False)
properties_df.head()

,ZIPcode,TaxRateArea_CITY,AIN,RollYear,TaxRateArea,AssessorID,PropertyLocation,PropertyType,PropertyUseCode,GeneralUseType,...,HouseFraction,StreetDirection,StreetName,UnitNo,City,ZIPcode5,rowID,CENTER_LAT,CENTER_LON,Location 1
0,91789-4036,DIAMOND BAR,8765017031,2019,10173,8765-017-031,20924 QUAIL RUN DR DIAMOND BAR CA 91789,SFR,0100,Residential,...,NaN,NaN,QUAIL RUN DR,NaN,DIAMOND BAR CA,91789.0,20198765017031,33.982467,-117.848366,"(33.98246687, -117.84836647)"
1,91789-4060,DIAMOND BAR,8765020011,2019,11950,8765-020-011,20923 E CANYON RIDGE LN DIAMOND BAR CA 91789,SFR,0100,Residential,...,NaN,E,CANYON RIDGE LN,NaN,DIAMOND BAR CA,91789.0,20198765020011,33.976384,-117.849724,"(33.97638402, -117.84972393)"
2,91789-4085,DIAMOND BAR,8765021098,2019,10328,8765-021-098,"20783 E CREST LN, #D WALNUT CA 91789",CND,010C,Residential,...,NaN,E,CREST LN,#D,WALNUT CA,91789.0,20198765021098,33.979641,-117.853396,"(33.97964088, -117.85339604)"
3,91789-4000,DIAMOND BAR,8765022038,2019,10190,8765-022-038,20740 E RIM LN DIAMOND BAR CA 91789,SFR,0100,Residential,...,NaN,E,RIM LN,NaN,DIAMOND BAR CA,91789.0,20198765022038,33.981122,-117.853957,"(33.98112249, -117.85395656)"
4,91765,DIAMOND BAR,8765023039,2019,10067,8765-023-039,20620 PEACEFUL WOODS DR DIAMOND BAR CA 91765,CND,01DC,Residential,...,NaN,NaN,PEACEFUL WOODS DR,NaN,DIAMOND BAR CA,91765.0,20198765023039,33.988432,-117.853275,"(33.988432, -117.85327457)"


In [24]:
# Read in parcels dataset and take a look at the head
parcels_df = pd.read_csv("Parcels.csv", low_memory=False)
parcels_df.head()

,OBJECTID,AIN,APN,SitusHouseNo,SitusFraction,SitusDirection,SitusUnit,SitusStreet,SitusAddress,SitusCity,SitusZIP,SitusFullAddress,TaxRateArea,TaxRateCity,AgencyClassNo,AgencyName,AgencyType,UseCode,UseCode_2,UseType,UseDescription,DesignType1,YearBuilt1,EffectiveYear1,Units1,Bedrooms1,Bathrooms1,SQFTmain1,DesignType2,YearBuilt2,EffectiveYear2,Units2,Bedrooms2,Bathrooms2,SQFTmain2,DesignType3,YearBuilt3,EffectiveYear3,Units3,Bedrooms3,Bathrooms3,SQFTmain3,DesignType4,YearBuilt4,EffectiveYear4,Units4,Bedrooms4,Bathrooms4,SQFTmain4,DesignType5,YearBuilt5,EffectiveYear5,Units5,Bedrooms5,Bathrooms5,SQFTmain5,Roll_Year,Roll_LandValue,Roll_ImpValue,Roll_PersPropValue,Roll_FixtureValue,Roll_HomeOwnersExemp,Roll_RealEstateExemp,Roll_PersPropExemp,Roll_FixtureExemp,Roll_LandBaseYear,Roll_ImpBaseYear,SpatialChangeDate,ParcelCreateDate,ParcelTypeCode,Assr_Map,Assr_Index_Map,QualityClass1,QualityClass2,QualityClass3,QualityClass4,QualityClass5,LegalDescLine1,LegalDescLine2,LegalDescLine3,LegalDescLine4,LegalDescLine5,LegalDescLineLast,LegalDescription,CENTER_LAT,CENTER_LON,CENTER_X,CENTER_Y,LAT_LON,ShapeSTArea,ShapeSTLength
0,1,7102032097,7102-032-097,16710.0,,,R 81,ORANGE AVE,16710 ORANGE AVE R 81,PARAMOUNT CA,90723-6896,16710 ORANGE AVE R 81 PARAMOUNT CA 90723,7354.0,PARAMOUNT,NaN,NaN,NaN,010C,01,Residential,Single,0133,1973.0,1973.0,1.0,2.0,2.0,1122.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019.0,107265.0,156476.0,0.0,0.0,0.0,0.0,0.0,0.0,2017.0,2017.0,NaN,0.0,1.0,7102-032,7102-NDX,D7A,NaN,NaN,NaN,NaN,TRACT NO 28568 CONDOMINIUM UNIT 81,NaN,NaN,NaN,NaN,NaN,TRACT NO 28568 CONDOMINIUM UNIT 81,33.882380,-118.175762,6.508318e+06,1.779624e+06,"33.882380, -118.175762",263890.580078,2386.488577
1,2,7102032107,7102-032-107,16710.0,,,NO U 91,ORANGE AVE,16710 ORANGE AVE NO U 91,PARAMOUNT CA,90723-7682,16710 ORANGE AVE NO U 91 PARAMOUNT CA 90723,7354.0,PARAMOUNT,NaN,NaN,NaN,010C,01,Residential,Single,0133,1973.0,1973.0,1.0,3.0,3.0,1361.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019.0,20660.0,114762.0,0.0,0.0,0.0,0.0,0.0,0.0,2000.0,2000.0,NaN,0.0,1.0,7102-032,7102-NDX,D7A,NaN,NaN,NaN,NaN,TRACT NO 28568 CONDOMINIUM UNIT 91,NaN,NaN,NaN,NaN,NaN,TRACT NO 28568 CONDOMINIUM UNIT 91,33.882380,-118.175762,6.508318e+06,1.779624e+06,"33.882380, -118.175762",263890.580078,2386.488577
2,3,7103002001,7103-002-001,8314.0,,,,ALONDRA BLVD,8314 ALONDRA BLVD,PARAMOUNT CA,90723-4404,8314 ALONDRA BLVD PARAMOUNT CA 90723,7273.0,PARAMOUNT,NaN,NaN,NaN,010V,01,Residential,Single,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019.0,23489.0,0.0,0.0,0.0,0.0,23489.0,0.0,0.0,1975.0,1975.0,NaN,0.0,0.0,7103-002,7103-NDX,NaN,NaN,NaN,NaN,NaN,CLEARWATER E 50 FT OF,NaN,NaN,NaN,NaN,LOT 1 BLK 59,CLEARWATER E 50 FT OF LOT 1 BLK 59,33.888934,-118.154216,6.514861e+06,1.781999e+06,"33.888934, -118.154216",4858.358398,294.421908
3,4,7103002003,7103-002-003,15922.0,,,,VIRGINIA AVE,15922 VIRGINIA AVE,PARAMOUNT CA,90723-5110,15922 VIRGINIA AVE PARAMOUNT CA 90723,7273.0,PARAMOUNT,NaN,NaN,NaN,0101,01,Residential,Single,0131,1961.0,1970.0,1.0,3.0,2.0,1881.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019.0,230330.0,217096.0,0.0,0.0,0.0,447426.0,0.0,0.0,2014.0,2014.0,NaN,0.0,0.0,7103-002,7103-NDX,D65B,NaN,NaN,NaN,NaN,CLEARWATER N 50 FT OF,NaN,NaN,NaN,NaN,LOT 3 BLK 59,CLEARWATER N 50 FT OF LOT 3 BLK 59,33.888456,-118.154379,6.514812e+06,1.781825e+06,"33.888456, -118.154379",7503.089844,400.122364
4,5,7103003005,7103-003-005,8216.0,,,,ALONDRA BLVD,8216 ALONDRA BLVD,PARAMOUNT CA,90723-4402,8216 ALONDRA BLVD PARAMOUNT CA 90723,7273.0,PARAMOUNT,NaN,NaN,NaN,2710,27,Commercial,Parking Lots (Commercial Use Properties),2700,2000.0,2000.0,0.0,0.0,0.0,7500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

## 0.2 Upload Core Datasets to pSQL

In [ ]:
# Upload data into pSQL using custom helper function
create_psql_schema(airBNB_df, "airbnb_full_listings")
create_psql_schema(properties_df, "Assessors_2019")
create_psql_schema(parcels_df, "parcels")

## 0.3 Basic SQL Queries for Understanding the Datasets

In [6]:
# Create connection cursor to pSQL
connection = connect(**params)
cursor = connection.cursor()

# Execute SQL query to test connection, dataset
cursor.execute("""SELECT * FROM "Assessors_2019" LIMIT 1;""")
cursor.fetchall()

[('91789-4036',
  'DIAMOND BAR',
  8765017031,
  2019,
  10173,
  '8765-017-031',
  '20924 QUAIL RUN DR  DIAMOND BAR CA  91789',
  'SFR',
  '0100',
  'Residential',
  'Single Family Residence',
  None,
  None,
  1,
  1986,
  1988,
  1901,
  3,
  3,
  1,
  19870721,
  114240.0,
  1986,
  195090.0,
  1986,
  309330.0,
  7000.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  'Y',
  309330.0,
  7000.0,
  302330.0,
  None,
  '06',
  6159,
  'TR=43162 LOT   48',
  20924,
  None,
  None,
  'QUAIL RUN DR',
  None,
  'DIAMOND BAR CA',
  91789.0,
  20198765017031,
  33.98246687,
  -117.84836647,
  '(33.98246687, -117.84836647)')]

In [21]:
# Execute SQL query to examine dataset features and feature types

query = """
SELECT * FROM parcels
WHERE "ShapeSTArea" >= 5000
LIMIT 3;
"""

df_parcels = pd.read_sql_query(query, connection)
df_parcels

,OBJECTID,AIN,APN,SitusHouseNo,SitusFraction,SitusDirection,SitusUnit,SitusStreet,SitusAddress,SitusCity,...,LegalDescLine5,LegalDescLineLast,LegalDescription,CENTER_LAT,CENTER_LON,CENTER_X,CENTER_Y,LAT_LON,ShapeSTArea,ShapeSTLength
0,1,7102032097,7102-032-097,16710.0,,,R 81,ORANGE AVE,16710 ORANGE AVE R 81,PARAMOUNT CA,...,None,None,TRACT NO 28568 CONDOMINIUM UNIT 81,33.882380,-118.175762,6.508318e+06,1.779624e+06,"33.882380, -118.175762",263890.580078,2386.488577
1,2,7102032107,7102-032-107,16710.0,,,NO U 91,ORANGE AVE,16710 ORANGE AVE NO U 91,PARAMOUNT CA,...,None,None,TRACT NO 28568 CONDOMINIUM UNIT 91,33.882380,-118.175762,6.508318e+06,1.779624e+06,"33.882380, -118.175762",263890.580078,2386.488577
2,4,7103002003,7103-002-003,15922.0,,,,VIRGINIA AVE,15922 VIRGINIA AVE,PARAMOUNT CA,...,None,LOT 3 BLK 59,CLEARWATER N 50 FT OF LOT 3 BLK 59,33.888456,-118.154379,6.514812e+06,1.781825e+06,"33.888456, -118.154379",7503.089844,400.122364


In [11]:
df_parcels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 91 columns):
OBJECTID                3 non-null int64
AIN                     3 non-null object
APN                     3 non-null object
SitusHouseNo            3 non-null float64
SitusFraction           3 non-null object
SitusDirection          3 non-null object
SitusUnit               3 non-null object
SitusStreet             3 non-null object
SitusAddress            3 non-null object
SitusCity               3 non-null object
SitusZIP                3 non-null object
SitusFullAddress        3 non-null object
TaxRateArea             3 non-null float64
TaxRateCity             3 non-null object
AgencyClassNo           0 non-null object
AgencyName              0 non-null object
AgencyType              0 non-null object
UseCode                 3 non-null object
UseCode_2               3 non-null object
UseType                 3 non-null object
UseDescription          3 non-null object
DesignType1     

In [10]:
df_parcels.transpose()

,0,1,2
OBJECTID,1,2,3
AIN,7102032097,7102032107,7103002001
APN,7102-032-097,7102-032-107,7103-002-001
SitusHouseNo,16710,16710,8314
SitusFraction,,,
SitusDirection,,,
SitusUnit,R 81,NO U 91,
SitusStreet,ORANGE AVE,ORANGE AVE,ALONDRA BLVD
SitusAddress,16710 ORANGE AVE R 81,16710 ORANGE AVE NO U 91,8314 ALONDRA BLVD
SitusCity,PARAMOUNT CA,PARAMOUNT CA,PARAMOUNT CA
